In [15]:
from facenet_pytorch import MTCNN
import torch
import cv2
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display, Video

# Determine if an NVIDIA GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

# Define MTCNN module
mtcnn = MTCNN(keep_all=True, device=device)

# Open video file
video_path = '19.mp4'
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
dim = (frame_width, frame_height)

frames_tracked = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB and PIL image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame_pil)
    
    # Draw faces
    frame_draw = frame_pil.copy()
    draw = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    # Convert back to OpenCV format and resize
    frame_tracked = np.array(frame_draw)
    frame_tracked = cv2.cvtColor(frame_tracked, cv2.COLOR_RGB2BGR)
    frame_tracked = cv2.resize(frame_tracked, (640, 360))
    
    frames_tracked.append(frame_tracked)

cap.release()

# Save tracked video
out_video_path = 'video_tracked1.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
video_tracked = cv2.VideoWriter(out_video_path, fourcc, fps, (640, 360))
for frame in frames_tracked:
    video_tracked.write(frame)
video_tracked.release()

# Display the tracked video
display(Video(out_video_path, width=640))


Running on device: cpu


In [6]:
from facenet_pytorch import InceptionResnetV1
import torch
from PIL import Image
import os
import numpy as np

# Determine if an NVIDIA GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the face recognition model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define the directory containing images of known faces
known_faces_dir = r'D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\data\test_images_aligned'  # Change this to your directory
known_face_names = []
known_face_embeddings = []

# Loop through each person in the directory
for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    
    if os.path.isdir(person_dir):  # Ensure it's a directory
        known_face_names.append(person_name)  # Add person's name

        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = Image.open(image_path).convert('RGB')  # Load and convert image
            image = image.resize((160, 160))  # Resize to (160, 160) for FaceNet
            image_tensor = torch.tensor(np.array(image)).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Compute embedding
            with torch.no_grad():
                embedding = model(image_tensor).cpu()
                known_face_embeddings.append(embedding)

# Convert list of embeddings to a tensor for easier comparison later
known_face_embeddings = torch.stack(known_face_embeddings)


In [5]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import cv2
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display, Video
import os

# Determine if an NVIDIA GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

# Define MTCNN module
mtcnn = MTCNN(keep_all=True, device=device)

# Load the face recognition model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define path to your embeddings folder
embeddings_folder = r'D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\examples\filept'  # Change this to your embeddings folder path
known_face_names = []
known_face_embeddings = []

# Load known face embeddings from the folder
for filename in os.listdir(embeddings_folder):
    if filename.endswith('.pt'):
        name = filename[:-3]  # Remove the .pt extension to get the name
        embedding_path = os.path.join(embeddings_folder, filename)
        embedding = torch.load(embedding_path).to(device)  # Load and move to device
        known_face_names.append(name)
        known_face_embeddings.append(embedding)

# Debug print to check the size of known embeddings
for idx, embedding in enumerate(known_face_embeddings):
    print(f"Embedding {known_face_names[idx]} shape: {embedding.shape}")

# Function to recognize faces
def recognize_face(embedding, known_face_embeddings, known_face_names, threshold=0.8):
    distances = [torch.norm(embedding - known_face_embedding).item() for known_face_embedding in known_face_embeddings]
    min_distance_index = np.argmin(distances)
    print(f"Distances: {distances}")  # Debug print
    if distances[min_distance_index] < threshold:
        return known_face_names[min_distance_index]
    return "Unknown"

# Open video file
video_path = '15.mp4'  # Path to your video file
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frames_tracked = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB and PIL image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame_pil)
    
    # Draw faces
    frame_draw = frame_pil.copy()
    draw = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
            # Get the face region for recognition
            face_region = frame_rgb[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            face_region = cv2.resize(face_region, (160, 160))  # Resize for FaceNet
            face_tensor = torch.tensor(face_region).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Get embedding and recognize
            with torch.no_grad():
                embedding = model(face_tensor).cpu()
                print(f"Current embedding shape: {embedding.shape}")  # Debug print
                identity = recognize_face(embedding, known_face_embeddings, known_face_names)

            print(f"Identified: {identity}")  # Debug print
            
            # Draw name on the frame (create a background for better visibility)
            text = identity
            text_bbox = draw.textbbox((box[0], box[1] - 10), text)  # Get the bounding box of the text
            draw.rectangle([text_bbox[0], text_bbox[1], text_bbox[2], text_bbox[3]], fill=(255, 0, 0))  # Draw background rectangle
            draw.text((box[0], box[1] - 10), text, fill=(255, 255, 255))  # Draw name above the box

    # Convert back to OpenCV format and resize
    frame_tracked = np.array(frame_draw)
    frame_tracked = cv2.cvtColor(frame_tracked, cv2.COLOR_RGB2BGR)
    frame_tracked = cv2.resize(frame_tracked, (640, 360))
    
    frames_tracked.append(frame_tracked)

cap.release()

# Save tracked video
out_video_path = 'video_tracked.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
video_tracked = cv2.VideoWriter(out_video_path, fourcc, fps, (640, 360))
for frame in frames_tracked:
    video_tracked.write(frame)
video_tracked.release()

# Display the tracked video
display(Video(out_video_path, width=640))


Running on device: cpu
Embedding Akada Iamong_embedding shape: torch.Size([1, 512])
Embedding Apiwat Rattanaphan_embedding shape: torch.Size([1, 512])
Embedding Apiwit Buachan_embedding shape: torch.Size([1, 512])
Embedding Ekapoj Suthiwong_embedding shape: torch.Size([1, 512])
Embedding Nattawat Thakhamho_embedding shape: torch.Size([1, 512])
Embedding Panupong Sitthiprom_embedding shape: torch.Size([1, 512])
Embedding Peerakarn Phraphinyokul_embedding shape: torch.Size([1, 512])
Embedding Phacharadanai Rossoda_embedding shape: torch.Size([1, 512])
Embedding Phurin Rueannimit_embedding shape: torch.Size([1, 512])
Embedding Sittikorn Thongdeenok_embedding shape: torch.Size([1, 512])
Embedding Thanaphum Hengarun_embedding shape: torch.Size([1, 512])
Embedding Thanapon Phetprapai_embedding shape: torch.Size([1, 512])
Current embedding shape: torch.Size([1, 512])
Distances: [1.3256340026855469, 1.269302487373352, 1.1831815242767334, 1.242719292640686, 1.3734323978424072, 1.218369841575622

KeyboardInterrupt: 

In [11]:
known_face_names 

[]

In [10]:
known_face_embeddings

tensor([[ 0.0430, -0.0596, -0.0278,  ..., -0.0368, -0.0273, -0.0660],
        [ 0.0479, -0.0599, -0.0065,  ..., -0.0566, -0.0555, -0.0496],
        [ 0.0101, -0.0197, -0.0148,  ..., -0.0644,  0.0151, -0.0436],
        ...,
        [-0.0396, -0.0635,  0.0135,  ..., -0.0466, -0.0233, -0.0844],
        [ 0.0121, -0.0422, -0.0076,  ...,  0.0134, -0.0427, -0.0213],
        [ 0.0312, -0.0216, -0.0049,  ..., -0.0577, -0.0461, -0.0355]])

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import cv2
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display, Video
import os

# Determine if an NVIDIA GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

# Define MTCNN module
mtcnn = MTCNN(keep_all=True, device=device)

# Load the face recognition model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define path to your embeddings folder
embeddings_folder = r'D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\examples\embeddings.pt'  # Change this to your embeddings folder path
known_face_names = []
known_face_embeddings = []

# Load known face embeddings from the folder
for filename in os.listdir(embeddings_folder):
    if filename.endswith('.pt'):
        name = filename[:-3]  # Remove the .pt extension to get the name
        embedding_path = os.path.join(embeddings_folder, filename)
        embedding = torch.load(embedding_path).to(device)  # Load and move to device
        known_face_names.append(name)
        known_face_embeddings.append(embedding)

# Convert list of embeddings to a tensor for easier comparison later
known_face_embeddings = torch.stack(known_face_embeddings)

# Function to recognize faces
def recognize_face(embedding, known_face_embeddings, known_face_names, threshold=0.8):
    distances = [torch.norm(embedding - known_face_embedding).item() for known_face_embedding in known_face_embeddings]
    min_distance_index = np.argmin(distances)
    if distances[min_distance_index] < threshold:
        return known_face_names[min_distance_index]
    return "Unknown"

# Open video file
video_path = '15.mp4'  # Path to your video file
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frames_tracked = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB and PIL image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame_pil)
    
    # Draw faces and names
    frame_draw = frame_pil.copy()
    draw = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)  # Draw rectangle around face
            # Get the face region for recognition
            face_region = frame_rgb[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            face_region = cv2.resize(face_region, (160, 160))  # Resize for FaceNet
            face_tensor = torch.tensor(face_region).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Get embedding and recognize
            with torch.no_grad():
                embedding = model(face_tensor).cpu()
                identity = recognize_face(embedding, known_face_embeddings, known_face_names)

            # Draw name on the frame
            text_position = (int(box[0]), int(box[1]) - 10)  # Position to draw the name above the rectangle
            draw.text(text_position, identity, fill=(255, 0, 0))  # Draw name

    # Convert back to OpenCV format and resize
    frame_tracked = np.array(frame_draw)
    frame_tracked = cv2.cvtColor(frame_tracked, cv2.COLOR_RGB2BGR)
    frame_tracked = cv2.resize(frame_tracked, (640, 360))
    
    frames_tracked.append(frame_tracked)

cap.release()

# Save tracked video
out_video_path = 'video_tracked2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
video_tracked = cv2.VideoWriter(out_video_path, fourcc, fps, (640, 360))
for frame in frames_tracked:
    video_tracked.write(frame)
video_tracked.release()

# Display the tracked video
display(Video(out_video_path, width=640))


Running on device: cpu


NotADirectoryError: [WinError 267] The directory name is invalid: 'D:\\Friend\\Super AI\\V89\\My Project\\facenet-pytorch-master\\examples\\embeddings.pt'

In [9]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import cv2
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display, Video

# Determine if an NVIDIA GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

# Define MTCNN module
mtcnn = MTCNN(keep_all=True, device=device)

# Load the face recognition model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Load embeddings from the file 'embeddings.pt'
embeddings_path = r'D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\examples\embeddings.pt'  # Change this to your file path
known_face_embeddings = torch.load(embeddings_path).to(device)  # Assume this is a tensor

# List of known face names (in the same order as the embeddings in the tensor)
known_face_names = []  # Update this with the actual names corresponding to embeddings

# Function to recognize faces
def recognize_face(embedding, known_face_embeddings, known_face_names, threshold=0.8):
    distances = [torch.norm(embedding - known_face_embedding).item() for known_face_embedding in known_face_embeddings]
    min_distance_index = np.argmin(distances)
    if distances[min_distance_index] < threshold:
        return known_face_names[min_distance_index]
    return "Unknown"

# Open video file
video_path = '2.mp4'  # Path to your video file
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frames_tracked = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB and PIL image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame_pil)
    
    # Draw faces and names
    frame_draw = frame_pil.copy()
    draw = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)  # Draw rectangle around face
            # Get the face region for recognition
            face_region = frame_rgb[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            face_region = cv2.resize(face_region, (160, 160))  # Resize for FaceNet
            face_tensor = torch.tensor(face_region).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Get embedding and recognize
            with torch.no_grad():
                embedding = model(face_tensor).cpu()
                identity = recognize_face(embedding, known_face_embeddings, known_face_names)

            # Draw name on the frame
            text_position = (int(box[0]), int(box[1]) - 10)  # Position to draw the name above the rectangle
            draw.text(text_position, identity, fill=(255, 0, 0))  # Draw name

    # Convert back to OpenCV format and resize
    frame_tracked = np.array(frame_draw)
    frame_tracked = cv2.cvtColor(frame_tracked, cv2.COLOR_RGB2BGR)
    frame_tracked = cv2.resize(frame_tracked, (640, 360))
    
    frames_tracked.append(frame_tracked)

cap.release()

# Save tracked video
out_video_path = 'video_tracked2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
video_tracked = cv2.VideoWriter(out_video_path, fourcc, fps, (640, 360))
for frame in frames_tracked:
    video_tracked.write(frame)
video_tracked.release()

# Display the tracked video
display(Video(out_video_path, width=640))


Running on device: cpu


In [18]:
from facenet_pytorch import InceptionResnetV1
import torch
from PIL import Image
import os
import numpy as np

# Determine if an NVIDIA GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the face recognition model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define the directory containing images of known faces
known_faces_dir = r'D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\data\test_images_aligned'  # Change this to your directory
known_face_names = []
known_face_embeddings = []

# Loop through each person in the directory
for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    
    if os.path.isdir(person_dir):  # Ensure it's a directory
        known_face_names.append(person_name)  # Add person's name

        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = Image.open(image_path).convert('RGB')  # Load and convert image
            image = image.resize((160, 160))  # Resize to (160, 160) for FaceNet
            image_tensor = torch.tensor(np.array(image)).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Compute embedding
            with torch.no_grad():
                embedding = model(image_tensor).cpu()
                known_face_embeddings.append(embedding)

# Convert list of embeddings to a tensor for easier comparison later
known_face_embeddings = torch.stack(known_face_embeddings)
# Function to recognize faces
def recognize_face(embedding, known_face_embeddings, known_face_names, threshold=0.8):
    distances = [torch.norm(embedding - known_face_embedding).item() for known_face_embedding in known_face_embeddings]
    min_distance_index = np.argmin(distances)
    if distances[min_distance_index] < threshold:
        return known_face_names[min_distance_index]
    return "Unknown"

# Open video file
video_path = '3.mp4'  # Path to your video file
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frames_tracked = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB and PIL image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame_pil)
    
    # Draw faces and names
    frame_draw = frame_pil.copy()
    draw = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)  # Draw rectangle around face
            # Get the face region for recognition
            face_region = frame_rgb[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            face_region = cv2.resize(face_region, (160, 160))  # Resize for FaceNet
            face_tensor = torch.tensor(face_region).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Get embedding and recognize
            with torch.no_grad():
                embedding = model(face_tensor).cpu()
                identity = recognize_face(embedding, known_face_embeddings, known_face_names)

            # Draw name on the frame
            # Draw name on the frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_position = (int(box[0]), int(box[1]) - 10)  # Position to draw the name above the rectangle
            cv2.putText(frame_tracked, identity, text_position, font, 0.8, (255, 0, 0), 2, cv2.LINE_AA)


    # Convert back to OpenCV format and resize
    frame_tracked = np.array(frame_draw)
    frame_tracked = cv2.cvtColor(frame_tracked, cv2.COLOR_RGB2BGR)
    frame_tracked = cv2.resize(frame_tracked, (640, 360))
    
    frames_tracked.append(frame_tracked)

cap.release()

# Save tracked video
out_video_path = 'video_tracked3.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
video_tracked = cv2.VideoWriter(out_video_path, fourcc, fps, (640, 360))
for frame in frames_tracked:
    video_tracked.write(frame)
video_tracked.release()

# Display the tracked video
display(Video(out_video_path, width=640))

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [33]:
from facenet_pytorch import InceptionResnetV1, MTCNN
import torch
from PIL import Image, ImageDraw
import cv2
import os
import numpy as np

# Determine if an NVIDIA GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the face recognition model
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)
mtcnn = MTCNN(keep_all=True, device=device)  # Initialize MTCNN for face detection

# Define the directory containing images of known faces
known_faces_dir = r'D:\Friend\Super AI\V89\My Project\facenet-pytorch-master\data\test_images_aligned'  # Change this to your directory
known_face_names = []
known_face_embeddings = []

# Loop through each person in the directory
for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    
    if os.path.isdir(person_dir):  # Ensure it's a directory
        known_face_names.append(person_name)  # Add person's name

        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = Image.open(image_path).convert('RGB')  # Load and convert image
            image = image.resize((160, 160))  # Resize to (160, 160) for FaceNet
            image_tensor = torch.tensor(np.array(image)).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
            
            # Compute embedding
            with torch.no_grad():
                embedding = model(image_tensor).cpu()
                known_face_embeddings.append(embedding)

# Convert list of embeddings to a tensor for easier comparison later
known_face_embeddings = torch.stack(known_face_embeddings)

# Function to recognize faces
def recognize_face(embedding, known_face_embeddings, known_face_names, threshold=1.0):
    distances = [torch.norm(embedding - known_face_embedding).item() for known_face_embedding in known_face_embeddings]
    min_distance_index = np.argmin(distances)
    print(f"Distances: {distances}") 
    if distances[min_distance_index] < threshold:
        return known_face_names[min_distance_index]
    return "Unknown"

# Open video file
video_path = '2.mp4'  # Path to your video file
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frames_tracked = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to RGB and PIL image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame_pil)
    
    # Draw faces and names
    frame_draw = frame_pil.copy()
    draw = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
        # Convert box coordinates to integer
            box = box.astype(int)
        # Draw rectangle around face using OpenCV
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)
        
        # Get the face region for recognition
            face_region = frame_rgb[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            face_region = cv2.resize(face_region, (160, 160))  # Resize for FaceNet
            face_tensor = torch.tensor(face_region).permute(2, 0, 1).float().unsqueeze(0).to(device) / 255.0
        
        # Get embedding and recognize
            with torch.no_grad():
                embedding = model(face_tensor).cpu()
                print(f"Current embedding shape: {embedding.shape}")
                identity = recognize_face(embedding, known_face_embeddings, known_face_names)
        
            print(f"Identified: {identity}")
        
        # Draw name on the frame
            text_position = (int(box[0]), int(box[1]) - 10)  # Position to draw the name above the rectangle
            cv2.putText(frame, identity, text_position, cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255, 0, 0), 2, cv2.LINE_AA)


            

    # Resize frame for output
    frame_tracked = cv2.resize(frame, (640, 360))
    frames_tracked.append(frame_tracked)

cap.release()

# Save tracked video
out_video_path = 'video_tracked5.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
video_tracked = cv2.VideoWriter(out_video_path, fourcc, fps, (640, 360))
for frame in frames_tracked:
    video_tracked.write(frame)
video_tracked.release()

# Display the tracked video
display(Video(out_video_path, width=640))


Current embedding shape: torch.Size([1, 512])
Distances: [1.326780915260315, 1.333076000213623, 1.3614587783813477, 1.3470032215118408, 1.363025188446045, 1.300369143486023, 1.311286211013794, 1.4173860549926758, 1.3790677785873413, 1.4023091793060303, 1.3307775259017944, 1.3263463973999023, 1.333714246749878, 1.3229399919509888, 1.3356091976165771, 1.2852085828781128, 1.3030695915222168, 1.355961561203003, 1.3216259479522705, 1.3878012895584106, 1.3270856142044067]
Identified: Unknown
Current embedding shape: torch.Size([1, 512])
Distances: [1.3377186059951782, 1.3363120555877686, 1.365031361579895, 1.3551883697509766, 1.3691855669021606, 1.3098164796829224, 1.322850227355957, 1.4259612560272217, 1.388258695602417, 1.4068241119384766, 1.3460469245910645, 1.3279014825820923, 1.3401379585266113, 1.3303840160369873, 1.3463068008422852, 1.2874234914779663, 1.30964195728302, 1.3617446422576904, 1.3290356397628784, 1.39802885055542, 1.3313701152801514]
Identified: Unknown
Current embedding 

KeyboardInterrupt: 

In [16]:
known_face_names

['Akada Iamong',
 'Akaradej Sukchan',
 'Apiwat Rattanaphan',
 'Apiwit Buachan',
 'Ekapoj Suthiwong',
 'Karanyapas Srikham',
 'Nattawat Thakhamho',
 'Noppawit Sabai',
 'Panupong Sitthiprom',
 'Peerakarn Phraphinyokul',
 'Phacharadanai Rossoda',
 'Phuri Khamfei',
 'Phurin Rueannimit',
 'Sittikorn Thongdeenok',
 'Thanaphum Hengarun',
 'Thanapon Phetprapai',
 'Thanwa Suktham']

In [34]:
known_face_embeddings

tensor([[[ 0.0183, -0.0476, -0.0655,  ..., -0.0362,  0.0060,  0.0063]],

        [[ 0.0222, -0.0672, -0.0749,  ..., -0.0052,  0.0130, -0.0061]],

        [[ 0.0288, -0.0677, -0.0765,  ..., -0.0008,  0.0098,  0.0027]],

        ...,

        [[-0.0040, -0.0670, -0.0284,  ..., -0.0302,  0.0043, -0.0202]],

        [[-0.0143, -0.0222, -0.0774,  ..., -0.0232,  0.0018, -0.0024]],

        [[ 0.0309, -0.0610, -0.1038,  ..., -0.0382,  0.0125,  0.0054]]])